In [1]:
import numpy as np
from matplotlib import pyplot as plt

import scipy.stats as scs
import scipy.signal as scsi

import psytrack as psy
import pandas as pd

import os
import datetime

%load_ext autoreload
%autoreload 2

import helperFns as mf
import plotFns as psyplt
import fitFns as ff

import pickle 

#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

#import nest_asyncio
#nest_asyncio.apply()

In [3]:
# Modified from Roy Et Al, 2021

IDs = ('GS027', 'GS028', 'GS029', 'GS030', 'GS037', 'GS040','JC025', 'JC028', 'JC029', 'JC039', 'JC044', 'JC047', 'JC048', 'JC052', 'JC057', 'JC059', 'JC061', 'JC062', 'JC067')

useVel = False
saveDat = True

weights = {'bias': 1,  # a special key
           's_high': 1,
           's_low': 1}

K = np.sum([weights[i] for i in weights.keys()])

hyper= {'sigInit': 2**4.,      # Set to a single, large value for all weights. Will not be optimized further.
        'sigma': [2**-4.]*K,   # Each weight will have it's own sigma optimized, but all are initialized the same
        'sigDay': 2**-4.}      # If NONE, then ignore session boundaries

optList = ['sigma','sigDay']

for ID in IDs:

    print('Starting ' + ID)
    
    keyword = "training"
    cutoff = 60
    numSessions = None # Use None if you want all sessions... counterintuitive, I know 
    trainingD = mf.getD(ID, keyword = keyword, cutoff = cutoff, sessionCutoff = numSessions, txt = None, useVel = useVel, untilTesting = True) #trainingD = psy.trim(trainingD, END = 500) 
    
    hyp, evd, wMode, hess_info = psy.hyperOpt(trainingD, hyper, weights, optList)

    psytrack = {
        'wMode': wMode,
        'weights': weights,
        'hyp': hyp,
    }
    trainingD['psytrack'] = psytrack

    today = datetime.datetime.now()
    date_time = today.strftime("%y%m%d")
    tmp = 'data/' + date_time + '/'

    if not os.path.isdir(tmp):
        os.makedirs(tmp)

    saveName = tmp + "{}_trainingDataBias.pkl".format(ID)
    with open(saveName, 'wb') as f:
        pickle.dump(trainingD, f)

Starting JC052
